In [44]:
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv("C:/Users/kisho/Datasets/twitter_new.csv",encoding = 'latin1',header=None)

In [46]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [47]:
df.columns = ['target','id','date','flag','user','text']

In [48]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [49]:
df.shape

(1600000, 6)

In [50]:
# df[df.isnull().any(axis=1)]
df.isna().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [51]:
df = df[['target','text']]

In [52]:
df['target'].value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [53]:
df['text'].apply(lambda x: True if x.startswith('@') else False).value_counts()

False    901921
True     698079
Name: text, dtype: int64

In [54]:
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [55]:
# df['text'] = df['text'].apply(lambda x: re.sub(r'^@[\w]*\s*', '', x))

In [56]:
# df[df['text'].apply(lambda x: True if x.startswith('@') else False)]

In [57]:
import re
def remove_pattern(text, pattern):
    r = re.findall(pattern, text)
    for word in r:
        text = re.sub(word, "", text)
    return text

In [58]:
df['text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")

In [59]:
df.head()

,target,text
0,0,"http://twitpic.com/2y1zl - Awww, that's a bum..."
1,0,is upset that he can't update his Facebook by ...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am..."


In [60]:
def remove_http(text):
    if text.startswith('http') or text.startswith(' http'):
        parts = text.split('-')
        return parts[1] if len(parts)>1 else ''
    else:
        return text

In [61]:
df['text'] = df['text'].apply(remove_http)

In [62]:
df['text'].head()

0     Awww, that's a bummer.  You shoulda got David...
1    is upset that he can't update his Facebook by ...
2     I dived many times for the ball. Managed to s...
3      my whole body feels itchy and like its on fire 
4     no, it's not behaving at all. i'm mad. why am...
Name: text, dtype: object

In [63]:
df['text'] = df['text'].apply(lambda x: x.lstrip() if x.startswith(' ') else x)

In [64]:
blanks = []
for i,target,text in df.itertuples():
    if type(text)==str and text.isspace() == True:
        blanks.append(i)
    elif len(text)<3:
        blanks.append(i)
len(blanks)

6736

In [65]:
df.shape

(1600000, 2)

In [66]:
df.drop(blanks,inplace = True)

In [67]:
df.shape

(1593264, 2)

In [68]:
x = df['text']
y = df['target']

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [70]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [71]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [72]:
text_clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [73]:
predictions = text_clf.predict(x_test)

In [74]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
confusion_matrix(y_test,predictions)

array([[187601,  51172],
       [ 47694, 191513]], dtype=int64)

In [75]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.80      0.79      0.79    238773
           4       0.79      0.80      0.79    239207

    accuracy                           0.79    477980
   macro avg       0.79      0.79      0.79    477980
weighted avg       0.79      0.79      0.79    477980



In [76]:
print(accuracy_score(y_test,predictions))

0.793158709569438


# Creating a new column polarity

### Created a new column polarity using vaderSentiment. Created the model using the polarity column as the output column.
### Got a better accuracy 

In [77]:
data = df

In [78]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
data['new']= data['text'].apply(lambda x: sid.polarity_scores(x))

In [79]:
def pol(x):
    if x['compound']<0:
        return 0
    elif x['compound']>0 :
        return 4
    else:
        return 2
data['polarity'] = data['new'].apply(pol)

In [80]:
x = data['text']
y = data['polarity']

In [81]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [82]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [83]:
text_clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [84]:
predictions = text_clf.predict(x_test)

In [85]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
confusion_matrix(y_test,predictions)

array([[105987,   2719,  17862],
       [  2797, 118117,   3680],
       [ 12839,   3012, 210967]], dtype=int64)

In [86]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85    126568
           2       0.95      0.95      0.95    124594
           4       0.91      0.93      0.92    226818

    accuracy                           0.91    477980
   macro avg       0.91      0.91      0.91    477980
weighted avg       0.91      0.91      0.91    477980



In [87]:
print(accuracy_score(y_test,predictions))

0.9102284614419013
